In [76]:
from lazyme import per_section
import nltk 

training_sentences = [[tuple(token.split('\t')) for token in sent] for sent in per_section(open('C:\\Test\\Training\\L3\\Train_L3.txt'))]
test_sentences = [[tuple(token.split('\t')) for token in sent] for sent in per_section(open('C:\\Test\\Testing\\L3\\Test_L3.txt'))]



In [77]:
print (training_sentences[0])

[('\ufeffالمادة', 'NC_DSF'), ('الثالثة', 'AS_DSF_13'), ('و', 'AS_DSF_23'), ('العشرون', 'AS_DSF_33'), (':', 'PUNC'), ('تحمي', 'VP_A3SF'), ('الدولة', 'NC_DSF'), ('عقيدة', 'NA_ISF'), ('الإسلام', 'NA_DSM'), ('،', 'PUNC'), ('و', 'RP'), ('تطبق', 'VP_A3SF'), ('شريعته', 'NA_ISF_R3RSRM'), ('،', 'PUNC'), ('و', 'RP'), ('تأمر', 'VP_A3SF'), ('ب', 'RP'), ('المعروف', 'AO_DSM'), ('و', 'RP'), ('تنهى', 'VP_A3SF'), ('عن', 'RP'), ('المنكر', 'AO_DSM'), ('،', 'PUNC'), ('و', 'RP'), ('تقوم', 'VP_A3SF'), ('ب', 'RP'), ('واجب', 'AS_ISM'), ('الدعوة', 'NA_DSF'), ('إلى', 'RP'), ('الله', 'NC_ISM'), ('.', 'PUNC')]


In [78]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        
        
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'prefix-4': sentence[index][:4],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        
        'prev_word1': '' if index == 0 else sentence[index - 1],
        
        'next_word1': '' if index == len(sentence) - 1 else sentence[index + 1],
        
        
        'is_numeric': sentence[index].isdigit(),
        
    }

In [79]:
from nltk.tag.util import untag
 
# Split the dataset for training and testing

 
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        X.append([features(untag(tagged), index) for index in range(len(tagged))])
        y.append([tag for _, tag in tagged])
 
    return X, y
 
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(test_sentences)
 
print(len(X_train))     
print(len(X_test))         
#print(X_train[0])
#print(y_train[0])

173
36


In [80]:
import time
start_time = time.time()

from sklearn_crfsuite import CRF
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
 
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
    c1=0.01,
    c2=0.01,
    max_iterations=100000,
    all_possible_transitions=True)
crf.fit(X_train, y_train)

print("--- %s seconds ---" % (time.time() - start_time))


--- 1031.9376163482666 seconds ---


In [81]:
labels = list(crf.classes_)

labels

['NC_DSF',
 'AS_DSF_13',
 'AS_DSF_23',
 'AS_DSF_33',
 'PUNC',
 'VP_A3SF',
 'NA_ISF',
 'NA_DSM',
 'RP',
 'NA_ISF_R3RSRM',
 'AO_DSM',
 'AS_ISM',
 'NA_DSF',
 'NC_ISM',
 'NA_ISM',
 'NC_DUM',
 'AA_DUM',
 'NA_ISF_R3RURM',
 'AS_ILM_R3RURM',
 'PR_SM',
 'VP_A3SM',
 'AS_ILM',
 'NA_DSM_AT',
 'NI_ILF_R3RLRM',
 'AS_DSM',
 'NA_ISM_R3RSRM',
 'RP_R3RSRM',
 'DL',
 'NA_ISM_R3RSRF',
 'NA_ILF_R3RSRF',
 'RP_R3RLRM',
 'VP_A3LM',
 'AC_DSF',
 'NA_DLF',
 'NA_DSF_AT',
 'PR_LF',
 'VP_A1LM',
 'NI_DSM',
 'PP_3LF',
 'NA_ILF',
 'NA_ILM_AT',
 'VS_A3LM',
 'NV_DLM',
 'NA_DSF_R3RLRM',
 'PR_SF',
 'VS_A3LM_R3RSRF',
 'AA_ISM_R2RSRM',
 'NA_IUM',
 'AC_DSM',
 'VP_A2SM',
 'AA_ISM',
 'PP_3SM',
 'VS_A3SM',
 'NI_ISM_R3RLRM',
 'IV',
 'VC_2SM_R1RSRM',
 'NC_ISF',
 'NI_ISM',
 'VP_A1SM',
 'NA_ISM_R1RSRM',
 'NC_DSM',
 'NC_ISM_R3RSRM',
 'VP_P3SM',
 'VP_A3SM_R3RSRM',
 'AO_ISM',
 'NI_ISM_R3RSRM',
 'VS_P3SF',
 'PD_SF',
 'NV_DLM_AT',
 'NA_ILM',
 'AO_ILF',
 'NV_DLF_AT',
 'NC_DLF_AT',
 'DIGIT',
 'NA_IUF',
 'NV_ILF',
 'NL_DSF_AT',
 'NC_ILF',
 

In [82]:
from sklearn_crfsuite import metrics
 
y_pred = crf.predict(X_test)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                  precision    recall  f1-score   support

          AA_DLF      0.000     0.000     0.000         2
          NA_DLF      0.449     0.537     0.489        41
       NA_DLF_AT      0.000     0.000     0.000         0
   NA_DLF_R1RLRM      0.000     0.000     0.000         0
          AA_DLM      0.800     0.800     0.800         5
          NA_DLM      0.000     0.000     0.000         0
       AA_DLM_AT      0.000     0.000     0.000         0
       NA_DLM_AT      0.000     0.000     0.000         2
          AA_DSF      0.667     0.400     0.500         5
          NA_DSF      0.380     0.675     0.486        40
       AA_DSF_AT      0.000     0.000     0.000         0
       NA_DSF_AT      0.579     0.786     0.667        14
   NA_DSF_R3RLRM      0.000     0.000     0.000         0
   NA_DSF_R3RSRM      0.000     0.000     0.000         0
          AA_DSM      0.667     0.429     0.522        14
          NA_DSM      0.534     0.807     0.643        88
       AA_DSM

In [83]:
import time
start_time = time.time()

from sklearn_crfsuite import metrics
 
y_pred = crf.predict(X_test)
print("Accuracy = ",metrics.flat_accuracy_score(y_test, y_pred))
print ("Recall =", metrics.flat_recall_score(y_test, y_pred, average='weighted',  labels=labels))
print ("Precision =", metrics.flat_precision_score(y_test, y_pred, average='weighted',  labels=labels))
print ("F1 =", metrics.flat_f1_score(y_test, y_pred, average='weighted',  labels=labels))

print("--- %s seconds ---" % (time.time() - start_time))

Accuracy =  0.7210736307580704


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Recall = 0.7324981577008106


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision = 0.7164077712984271


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


F1 = 0.709654089812854
--- 0.9565014839172363 seconds ---


In [84]:
y_pred = crf.predict(X_test)
print ("F1 =", metrics.flat_f1_score(y_test, y_pred, average='weighted',  labels=labels))

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


F1 = 0.709654089812854


In [85]:
sentence = 'من هذا الرجل ؟'.split()
 
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))

[('من', 'RP'), ('هذا', 'PD_SM'), ('الرجل', 'NC_DSM'), ('؟', 'PUNC')]


In [86]:
sentence = ' تبدأ الرواية في أغسطس 1989 في بيروت حيث تدور طواحين الحرب الأهلية اللبنانية على دانيا الطالبة في المرحلة الثانوية و أسرتها . لأواء الحرب و القذائف الطائشة تدفع ب سكان البناية إلى التجمع في ملجأ واحد ، و هنا تعرض لنا الروائية كيف تذوب الفروقات بين السكّان ، ل يظهر الخيط الإنساني الرفيع الذي يشبكهم . يتقاسمون الاحتياجات ، و يتشاطرون الخوف و القلق . في خضم ذلك ، تتوثق علاقة دانيا ب جارهم رياض الصحافي صاحب المبادئ الذي يلحظ موهبة دانيا في الكتابة و يتبناها ثقافيا . يتفرّق معظم قاطني الملجأ ، إما ب الموت ، أو ب الرحيل إلى أماكن آمنة . يحصل والد دانيا على عرض من الشركة التي كان يعمل فيها ل ينضم ل فرعها في الكويت .'.split()
 
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))

[('تبدأ', 'VP_A3SM'), ('الرواية', 'NA_DSF'), ('في', 'RP'), ('أغسطس', 'NC_ISM'), ('1989', 'DIGIT'), ('في', 'RP'), ('بيروت', 'VS_A3SF'), ('حيث', 'DL'), ('تدور', 'VP_A3SF'), ('طواحين', 'NI_ISM'), ('الحرب', 'NA_DSF'), ('الأهلية', 'NA_DSF_AT'), ('اللبنانية', 'NA_DSF_AT'), ('على', 'RP'), ('دانيا', 'AS_ISM'), ('الطالبة', 'NA_DSF'), ('في', 'RP'), ('المرحلة', 'NA_DSF'), ('الثانوية', 'NC_DSF_AT'), ('و', 'RP'), ('أسرتها', 'NA_ILF_R3RSRF'), ('.', 'PUNC'), ('لأواء', 'NA_ISM'), ('الحرب', 'NA_DSM'), ('و', 'RP'), ('القذائف', 'NC_DLF'), ('الطائشة', 'NA_DLF'), ('تدفع', 'VP_A3LF'), ('ب', 'RP'), ('سكان', 'NA_ISM'), ('البناية', 'NA_DSF'), ('إلى', 'RP'), ('التجمع', 'NA_DSM'), ('في', 'RP'), ('ملجأ', 'NI_ISM'), ('واحد', 'NI_ISM'), ('،', 'PUNC'), ('و', 'RP'), ('هنا', 'PD'), ('تعرض', 'NA_ISM'), ('لنا', 'RP_R1RLRM'), ('الروائية', 'NA_DSF_AT'), ('كيف', 'RP'), ('تذوب', 'NA_ISM'), ('الفروقات', 'NA_DLF'), ('بين', 'NI_ISM'), ('السكّان', 'NC_DSM'), ('،', 'PUNC'), ('ل', 'RP'), ('يظهر', 'VP_A3SM'), ('الخيط', 'NC_DSM'), 

In [94]:
sentence =' الرياض عاصمتنا الحبيبة هي عاصمة القرار العربي ب امتياز ، ف منها تصدر الخطوط العريضة ل رسم السياسات التي تكفل تحقيق المصالح العربية على مختلف ملفاتها، في هذا الأسبوع تشهد الرياض - ك ما هي عادتها - أحداثاً بالغة الأهمية ب انعقاد قمة مجلس التعاون و الاحتفال ب الذكرى الرابعة ل تولي خادم الحرمين الشريفين الملك سلمان بن عبدالعزيز - حفظه الله - مقاليد الحكم الذي شهدت البلاد في عهده نقلات نوعية غير مسبوقة انعكست إيجاباً على كل مخرجات التنمية الاقتصادية و الاجتماعية من خلال رؤية 2030 .'.split()
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))

[('الرياض', 'NA_DSM'), ('عاصمتنا', 'RP_R3RSRF'), ('الحبيبة', 'NA_DSF'), ('هي', 'PP_3SF'), ('عاصمة', 'AS_ISF'), ('القرار', 'NA_DSM'), ('العربي', 'NV_DSM_AT'), ('ب', 'RP'), ('امتياز', 'NA_ISM'), ('،', 'PUNC'), ('ف', 'RP'), ('منها', 'RP_R3RSRF'), ('تصدر', 'VP_A3SF'), ('الخطوط', 'NC_DSF'), ('العريضة', 'AA_DSF'), ('ل', 'RP'), ('رسم', 'NA_ISM'), ('السياسات', 'NA_DLF'), ('التي', 'PR_SF'), ('تكفل', 'VS_A3SM'), ('تحقيق', 'NA_ISM'), ('المصالح', 'NC_DSM'), ('العربية', 'NV_DSF_AT'), ('على', 'RP'), ('مختلف', 'AO_ISM'), ('ملفاتها،', 'AO_ISM'), ('في', 'RP'), ('هذا', 'PD_SM'), ('الأسبوع', 'NC_DSM'), ('تشهد', 'NA_ISM'), ('الرياض', 'NA_DSM'), ('-', 'PUNC'), ('ك', 'RP'), ('ما', 'RP'), ('هي', 'PP_3LF'), ('عادتها', 'NA_ILF_R3RSRF'), ('-', 'PUNC'), ('أحداثاً', 'NC_ISM'), ('بالغة', 'NA_ISF'), ('الأهمية', 'NA_DSF'), ('ب', 'RP'), ('انعقاد', 'NA_ISM'), ('قمة', 'NA_ISF'), ('مجلس', 'NL_ISM'), ('التعاون', 'NA_DSM'), ('و', 'RP'), ('الاحتفال', 'NA_DSM'), ('ب', 'RP'), ('الذكرى', 'NA_DSF'), ('الرابعة', 'NA_DSF'), ('ل'

In [88]:
sentence ='أحبطت القوات الأفغانية اليوم ، محاولة مسلحين ل تنفيذ هجوم صاروخي على العاصمة كابول . و ذكر بيان صادر عن شرطة كابول نقلته وكالة أنباء " خاما برس " الأفغانية ، أن قوات الشرطة اكتشفت صاروخًا من طراز بي إم - 1 في محيط منطقة الشرطة التاسعة في المدينة صباح اليوم . و أضاف البيان أن الصاروخ تم اكتشافه في منطقة دشت بادولا ، و تم إبطال مفعوله من قبل فريق شرطة التخلص من المتفجرات ، و لم تعلق الجماعات المسلحة المناهضة للحكومة و من بينها حركة طالبان ب شأن الحادث حتى الآن'.split()
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))

[('أحبطت', 'VS_A3SF'), ('القوات', 'NA_DLF'), ('الأفغانية', 'NC_DSF_AT'), ('اليوم', 'NI_DSM'), ('،', 'PUNC'), ('محاولة', 'NA_ISF'), ('مسلحين', 'AS_ILM'), ('ل', 'RP'), ('تنفيذ', 'VP_A3SF'), ('هجوم', 'NA_ISM'), ('صاروخي', 'AS_ISM'), ('على', 'RP'), ('العاصمة', 'AS_DSF'), ('كابول', 'RP'), ('.', 'PUNC'), ('و', 'RP'), ('ذكر', 'NA_ISM'), ('بيان', 'NA_ISM'), ('صادر', 'AS_ISM'), ('عن', 'RP'), ('شرطة', 'NA_ISF'), ('كابول', 'RP'), ('نقلته', 'NA_ISF_R3RSRM'), ('وكالة', 'NA_ISF'), ('أنباء', 'NA_ILF'), ('"', 'PUNC'), ('خاما', 'RP'), ('برس', 'NC_ISM'), ('"', 'PUNC'), ('الأفغانية', 'NA_DSF'), ('،', 'PUNC'), ('أن', 'RP'), ('قوات', 'NA_ILF'), ('الشرطة', 'NA_DSF'), ('اكتشفت', 'VS_A3LF'), ('صاروخًا', 'AS_ISM'), ('من', 'RP'), ('طراز', 'NA_ISM'), ('بي', 'NC_ISM'), ('إم', 'NC_ISM'), ('-', 'PUNC'), ('1', 'DIGIT'), ('في', 'RP'), ('محيط', 'NA_ISM'), ('منطقة', 'NA_ISF'), ('الشرطة', 'NA_DSF'), ('التاسعة', 'NA_DSF'), ('في', 'RP'), ('المدينة', 'NC_DSM'), ('صباح', 'NA_ISM'), ('اليوم', 'NI_DSM'), ('.', 'PUNC'), ('و', 

In [89]:
sentence ='الله لا إله إلا هو الحي القيوم لا تأخذه سنة و لا نوم له ما في السماوات و ما في الأرض من ذا الذي يشفع عنده إلا ب إذنه يعلم ما بين أيديهم و ما خلفهم و لا يحيطون ب شيء من علمه إلا ب ما شاء وسع كرسيه السماوات و الأرض و لا يئوده حفظهما و هو العلي العظيم'.split()
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))

[('الله', 'NC_ISM'), ('لا', 'RP'), ('إله', 'NC_ISM'), ('إلا', 'RP'), ('هو', 'PP_3SM'), ('الحي', 'NA_DSM'), ('القيوم', 'NC_DSM'), ('لا', 'RP'), ('تأخذه', 'VP_A2SM'), ('سنة', 'NI_ISF'), ('و', 'RP'), ('لا', 'RP'), ('نوم', 'VP_A1LM'), ('له', 'RP_R3RSRM'), ('ما', 'PR_SM'), ('في', 'RP'), ('السماوات', 'NC_DLF'), ('و', 'RP'), ('ما', 'PR_SM'), ('في', 'RP'), ('الأرض', 'NC_DSM'), ('من', 'RP'), ('ذا', 'PD_SM'), ('الذي', 'PR_SM'), ('يشفع', 'VP_A3SM'), ('عنده', 'NA_ISM_R3RSRM'), ('إلا', 'RP'), ('ب', 'RP'), ('إذنه', 'NA_ISM_R3RSRM'), ('يعلم', 'VP_A3SM'), ('ما', 'PR_SM'), ('بين', 'NI_ISM'), ('أيديهم', 'RP_R3RLRM'), ('و', 'RP'), ('ما', 'PR_SM'), ('خلفهم', 'VS_A3SM'), ('و', 'RP'), ('لا', 'RP'), ('يحيطون', 'VP_A3LM'), ('ب', 'RP'), ('شيء', 'NA_ISM'), ('من', 'RP'), ('علمه', 'VS_A3SM_R3RSRM'), ('إلا', 'RP'), ('ب', 'RP'), ('ما', 'PR_SM'), ('شاء', 'VS_A3SM'), ('وسع', 'NA_ISM'), ('كرسيه', 'NA_ISM_R3RSRM'), ('السماوات', 'NC_DLF'), ('و', 'RP'), ('الأرض', 'NC_DSM'), ('و', 'RP'), ('لا', 'RP'), ('يئوده', 'VP_A3SM_R

In [90]:


sentence ='تستخدم مواد التراث الشعبي و الحياة الشعبية في إعادة بناء الفترات التاريخية الغابرة ل الأمم والشعوب و التي لا يوجد لها إلا شواهد ضئيلة متفرقة و تستخدم أيضا ل إبراز الهوية الوطنية و القومية و الكشف عن ملامحها  التراث والمأثورات التراثية ب شكلها و مضمونها أصيلة و متجذرة إلا أن فروعها تتطور و تتوسع مع مرور الزمن و بنسب مختلفة و ذلك ب فعل التراكم الثقافي و الحضاري و تبادل التأثر و التأثير مع الثقافات و الحضارات الأخرى و عناصر التغيير و الحراك في الظروف الذاتية و الاجتماعية ل كل مجتمع.  ويتنوع التراث ب اختلاف ما تحمله الجذور إلى الشجرة . ف قد تحمل إليها قوتها المتمثل في الأملاح المعدنية و هو ب مثابة ما دون من التراث ف إن فقد ف ستصير الأمة ك شجرة حبست عنها الأملاح المعدنية ف ستذبل حتما شيئا ف شيئا ثم تضمحل . و قد تحمل الجذور الماء ف تتناقله مكونات الشجرة ل يشربوا منه واحدا تلو الآخر ف يشرب كل سلف و يسلم الماء ل خلفه شأن تواترته الأجيال أبا عن جد من تراث شفوي ك الأمثال الشعبية و الحكايات الهادفة و غيرهما . '.split()
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))

[('تستخدم', 'VS_A3SM'), ('مواد', 'NA_ISM'), ('التراث', 'NA_DSM'), ('الشعبي', 'NC_DSM_AT'), ('و', 'RP'), ('الحياة', 'NA_DSF'), ('الشعبية', 'NC_DSF_AT'), ('في', 'RP'), ('إعادة', 'NA_ISF'), ('بناء', 'NA_ISM'), ('الفترات', 'NA_DLF'), ('التاريخية', 'NA_DSF_AT'), ('الغابرة', 'NA_DSF'), ('ل', 'RP'), ('الأمم', 'NC_DSM'), ('والشعوب', 'NC_ISM'), ('و', 'RP'), ('التي', 'PR_LF'), ('لا', 'RP'), ('يوجد', 'VP_P3SM'), ('لها', 'RP_R3RSRF'), ('إلا', 'RP'), ('شواهد', 'NC_ISM'), ('ضئيلة', 'NA_ISF'), ('متفرقة', 'AS_ISF'), ('و', 'RP'), ('تستخدم', 'VS_A3SM'), ('أيضا', 'NA_ISM'), ('ل', 'RP'), ('إبراز', 'NA_ISM'), ('الهوية', 'NA_DSF'), ('الوطنية', 'NA_DSF_AT'), ('و', 'RP'), ('القومية', 'NA_DSF'), ('و', 'RP'), ('الكشف', 'NC_DSM'), ('عن', 'RP'), ('ملامحها', 'NA_ISM_R3RSRF'), ('التراث', 'NA_DSM'), ('والمأثورات', 'NA_ILF'), ('التراثية', 'NA_DSF'), ('ب', 'RP'), ('شكلها', 'NA_ISM_R3RSRF'), ('و', 'RP'), ('مضمونها', 'RP_R3RSRF'), ('أصيلة', 'NA_ISF'), ('و', 'RP'), ('متجذرة', 'AS_ISF'), ('إلا', 'RP'), ('أن', 'RP'), ('فرو

In [91]:
print("Number of Transition Features ")
len(crf.transition_features_)

Number of Transition Features 


4059

In [92]:
from collections import Counter

print("Number of State Features ",len(crf.state_features_))
Counter(crf.transition_features_).most_common(107)

Number of State Features  30585


[(('NC_ISM_12', 'NC_ISM_22'), 9.615273),
 (('FOREIGN', 'FOREIGN'), 5.463013),
 (('NC_ISF_12', 'NC_ISF_22'), 5.02872),
 (('PR_SM', 'VS_A3SM_R3RSRM'), 3.866264),
 (('AS_DSF_13', 'AS_DSF_23'), 3.821782),
 (('NC_DLF', 'AA_DLF'), 3.674932),
 (('AS_DSF_23', 'AS_DSF_33'), 3.407205),
 (('VS_P3SF', 'NV_ILF'), 3.366006),
 (('NC_ISM', 'NC_ISM'), 3.346888),
 (('PR_LM', 'NI_ISM_R3RSRM'), 3.253593),
 (('VS_A3LM', 'AS_ILM'), 3.246004),
 (('NC_ILF', 'AA_ILF'), 3.23225),
 (('NC_DSF', 'ABBREV'), 3.137945),
 (('PR_SM', 'VS_A3SM'), 3.109768),
 (('SYMB', 'DIGIT'), 3.086723),
 (('NC_DSF', 'AS_DSF_13'), 3.082508),
 (('NA_DSF_AT', 'AA_DSF'), 3.069833),
 (('AA_ISM', 'AA_ISM'), 3.061853),
 (('VS_A3LF_R1RLRM', 'NC_ILF'), 2.997821),
 (('VS_P3SM', 'RP_R3RSRF'), 2.992278),
 (('DIGIT', 'ABBREV'), 2.964949),
 (('NI_DLF', 'NI_DSF'), 2.951435),
 (('NI_DSM', 'NI_DSM'), 2.928289),
 (('PR_LM', 'VP_A3SF_R3RLRM'), 2.876215),
 (('NC_ILF', 'VP_A3SF'), 2.863269),
 (('NC_ISM', 'PUNC'), 2.836076),
 (('PR_SM', 'VP_P3SM'), 2.81338